In [1]:
import itertools

In [2]:
m = Matrix(ZZ, [[1, 0, 0], [0x5DEECE66D, 2**48, 0], [205749139540585, 0, 2**48]])
m

[              1               0               0]
[    25214903917 281474976710656               0]
[205749139540585               0 281474976710656]

In [3]:
sm = m.transpose().LLL().transpose()
sm

[ 1270789291  2355713969  3756485696]
[-2446815537 -1026597795  2345310016]
[ 2154219555 -4110294631  2015749696]

In [4]:
eq1 = (193514046488576, 211106232532991)
eq2 = (79164837199861, 81363860455412)
eq3 = (87683566278982, 105275752323397)
in_eqs = [eq1, eq2, eq3]

In [5]:
# Find all corners of the box we are searching in
# 
eqs = [(ceil(min(*row)), floor(max(*row))) for row in (m.inverse()*matrix(itertools.product(*in_eqs)).transpose()).rows()]

In [6]:
eqs

[(193514046488576, 211106232532991),
 (-18911177937, -17335246443),
 (-154311854655437, -141452533434152)]

In [7]:
(eqs[0][1]-eqs[0][0])*(eqs[1][1]-eqs[1][0])*(eqs[2][1]-eqs[2][0])

356512850743692143396951339010422850

In [8]:
eqs = [(ceil(min(*row)), floor(max(*row))) for row in (sm.inverse()*matrix(itertools.product(*in_eqs)).transpose()).rows()]
eqs

[(8141, 12464), (2534, 7606), (45192, 49191)]

In [9]:
(eqs[0][1]-eqs[0][0]+1)*(eqs[1][1]-eqs[1][0]+1)*(eqs[2][1]-eqs[2][0]+1)

87742608000

In [11]:
import time
import numpy as np
from tqdm.notebook import tqdm
it = itertools.product(*[range(eq[0],eq[1]+1) for eq in eqs])
elems = 87742608000
#it = itertools.islice(it, elems)
conv = np.matrix(sm)
t1 = time.monotonic()
lower_bounds = [(a[0],) for a in in_eqs]
upper_bounds = [(a[1],) for a in in_eqs]
results = []
with open("output.txt", "w") as f:
    for p in tqdm(itertools.batched(it, n=2048), total=int(elems // 2048)):
        p = np.matrix(p).transpose()
        seeds = conv*p
        res1 = np.less_equal(lower_bounds, seeds).all(axis=0)
        res2 = np.less_equal(seeds, upper_bounds).all(axis=0)
        res = np.vstack([res1, res2]).all(axis=0)
        if res.any():
            row = seeds.T[res.flat]
            results.extend(row)
            if len(results) > 1024:
                f.write(str(results) + "\n")
                results.clear()

  0%|          | 0/42843070 [00:00<?, ?it/s]

KeyboardInterrupt: 